!pip install -U tqdm
!pip install -U nltk
!pip install -U spacy
!pip install -U numpy
!pip install -U pandas
!pip install -U sklearn
!pip install -U torch
!pip install -U torchtext
!python -m spacy download en_core_web_md

In [2]:
import pandas as pd

data = pd.read_csv('/Users/wenjiazhai/Google Drive/Colab Notebooks/datasets/IMDB Dataset.csv')
print(len(data))
print(data.head())

50000
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [3]:
from tqdm import tqdm
import nltk

from nltk.stem import WordNetLemmatizer
# nltk.download()
lemmatizer = WordNetLemmatizer()

import re, spacy
nlp = spacy.load('en_core_web_md')

In [4]:
# print(re.sub('<.+?>', '', data.review[17]))

In [5]:
processed_review = []
sentiment = []
word2id = {'<PAD>':0}
id2word = {0:'<PAD>'}
vocab = set('<PAD>')
count = 1
SEQ_LEN = 100

for i in tqdm(range(len(data))):
    text = data.review[i].lower()
    text = re.sub('<.+?>', '', text)
    text = re.sub('[<>]', '', text)
    text = [lemmatizer.lemmatize(token.text) for token in nlp.tokenizer(text)][:SEQ_LEN]

    tmp = [0] * (SEQ_LEN - len(text)) if len(text) < SEQ_LEN else []
        
    for word in text:
        if word not in vocab:
            vocab.add(word)
            word2id[word] = count
            tmp.append(count)
            count += 1
        else:
            tmp.append(word2id[word])

    processed_review.append(tmp)
    
    if data.sentiment[i] == 'positive':
        sentiment.append(1)
    elif data.sentiment[i] == 'negative':
        sentiment.append(0)
    
id2word = {val:key for key, val in word2id.items()}
print(processed_review[0], sentiment[0])

100%|██████████| 50000/50000 [01:31<00:00, 545.91it/s]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 8, 34, 35, 36, 13, 37, 38, 39, 40, 41, 42, 2, 43, 23, 44, 45, 46, 22, 47, 3, 48, 49, 19, 50, 35, 23, 25, 26, 51, 24, 52, 53, 3, 54, 55, 56, 57, 19, 25, 52, 58, 59, 60, 30, 61, 62, 63, 23, 64, 56, 43, 19, 38, 26, 65, 23, 46, 3, 66, 67, 2, 3, 68, 26, 69, 13, 24] 1


In [6]:
# len(sentiment)

50000

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def clean(text):
    text = re.sub('<.+?>', '', text)
    text = [lemmatizer.lemmatize(token.text) for token in nlp.tokenizer(text)]
    return ' '.join(text)

data['processed_review'] = data.review.apply(clean)

data.processed_review[0]

from torchtext.data.import Field

tokenizer = lambda x: x.split()

# def tokenizer(text):
#     return [token.text for token in nlp.tokenizer(text)]
TEXT = Field(sequential=True, tokenize=tokenizer, lower=True)
LABEL = Field(sequential=False, use_vocab=False)

padded_review = []

for review in processed_review:
    if len(review) < 100:
        padded_review.append(['<PAD>'] * (100 - len(review)))
    else:
        padded_review.append(review[:100])

In [7]:
# print(len(processed_review))

In [8]:
from sklearn.model_selection import train_test_split

# review = data.processed_review.tolist()
# sentiment = data.sentiment

train, X_test, label, y_test = train_test_split(processed_review, sentiment, train_size=0.9, random_state=1988)
X_train, X_val, y_train, y_val = train_test_split(train, label, train_size=0.9, random_state=1988)

# train = pd.DataFrame({'review':X_train, 'sentiment': y_train})
# val = pd.DataFrame({'review':X_val, 'sentiment':y_val})
# test = pd.DataFrame({'review':X_test, 'sentiment':y_test})

# train.to_csv('/Users/wenjiazhai/Google Drive/Colab Notebooks/datasets/IMBD train.csv')
# val.to_csv('/Users/wenjiazhai/Google Drive/Colab Notebooks/datasets/IMDB val.csv')
# test.to_csv('/Users/wenjiazhai/Google Drive/Colab Notebooks/datasets/IMDB test.csv')

In [9]:
# print(X_train[0])
# print(y_train[0])

In [10]:
from torch.utils.data import TensorDataset, DataLoader
import torch

BATCH_SIZE = 64

train_ds = TensorDataset(torch.as_tensor(X_train), torch.as_tensor(y_train))
val_ds = TensorDataset(torch.as_tensor(X_val), torch.as_tensor(y_val))
test_ds = TensorDataset(torch.as_tensor(X_test), torch.as_tensor(y_test))

train_iter = DataLoader(train_ds, batch_size=BATCH_SIZE, drop_last=True) # (BATCH_SIZE, SEQ_LEN)
val_iter = DataLoader(val_ds, batch_size=BATCH_SIZE, drop_last=True)
test_iter = DataLoader(test_ds, batch_size=BATCH_SIZE, drop_last=True)

In [11]:
# len(vocab)

In [12]:
from torch import nn, optim
from torch.nn import functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embed_size, dropout, batch_size):
        super(CNN, self).__init__()
        self.batch_size = batch_size
        
        self.embedding = nn.Embedding(vocab_size, embed_size) # (BATCH_SIZE, SEQ_LEN, embed_size)
        self.conv1 = nn.Conv2d(1, 8, 5)
        self.pool1 = nn.MaxPool2d(4)
        self.conv2 = nn.Conv2d(8, 1, 5)
        self.pool2 = nn.MaxPool2d(4)
#         self.conv3 = nn.Conv2d(16, 1, 5)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(25, 1)

    def forward(self, x):
        x = self.embedding(x)
        x.unsqueeze_(1) # (BATCH_SIZE, 1, SEQ_LEN, embed_size)
        x = self.conv1(x)
        x = F.relu(self.pool1(x))
        x = self.dropout(x)
        x = self.conv2(x)
        x = F.relu(self.pool2(x))
#         print(x.shape)
        x = self.dropout(x)
        return self.out(x.view(self.batch_size, -1))

In [13]:
EMBED_SIZE = 100
DROPOUT = 0.5
model = CNN(len(vocab), EMBED_SIZE, DROPOUT, BATCH_SIZE)

In [14]:
model

CNN(
  (embedding): Embedding(90175, 100)
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 1, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (out): Linear(in_features=25, out_features=1, bias=True)
)

In [15]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [16]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [17]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for text, label in iterator:
        optimizer.zero_grad()
        preds = model(text)
        loss = criterion(preds.squeeze(), label.float())
        acc = binary_accuracy(preds.squeeze(), label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for text, label in iterator:
            preds = model(text)
            loss = criterion(preds.squeeze(), label.float())
            acc = binary_accuracy(preds.squeeze(), label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - elapsed_mins * 60)
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 40s
	Train Loss: 0.693 | Train Acc: 49.95%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 02 | Epoch Time: 1m 43s
	Train Loss: 0.693 | Train Acc: 49.92%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 03 | Epoch Time: 1m 38s
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 04 | Epoch Time: 1m 46s
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 05 | Epoch Time: 1m 40s
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 06 | Epoch Time: 1m 41s
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 07 | Epoch Time: 1m 40s
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 08 | Epoch Time: 1m 41s
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 09 | Epoch Time: 1m 40s
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.693 |  Val. Acc: 49.58%
Epoch: 10 | Epoch T